# Federated learning with YOLOv8

## Before you start

Let's make sure that you have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

## Install the ultralytics library

In [ ]:
!pip install ultralytics

## Download the dataset

Clone the project from GitHub

In [ ]:
!git clone https://github.com/

Extract the datasets folder

In [ ]:
!mv /content/project/datasets /content/

## Train the model

Configure the run and train the model

In [1]:
%cd /content

from ultralytics import YOLO
import copy
import torch
from numpy import random
import os

# Configure the run
PROJECT = 'augmented1500_random'
DATASET = 'augmented1500'
GLOBAL_MODEL_EPOCHS = 5
FEDERATED = True
GLOBAL_FEDERATED_EPOCHS = 5
CLIENT_EPOCHS = 20
TOTAL_FEDERATED_CLIENTS = 5
ACTIVE_FEDERATED_CLIENTS = 3
BEST_MAPS = 3
GLOBAL_AVG_EPOCHS = 1

# Define the averaging function
def average_weights(w):
  w_avg = copy.deepcopy(w[0])
  for key in w_avg.keys():
      for i in range(1, len(w)):
          w_avg[key] += w[i][key]
      w_avg[key] = torch.div(w_avg[key], len(w))
  return w_avg

def main():
  # Check if the project already exists
  if os.path.exists('/content/runs/' + PROJECT):
    raise Exception(f'Project {PROJECT} already exists, choose a new project name')

  # Print the run configuration
  print('PROJECT : ' + PROJECT)
  print('DATASET : ' + DATASET)
  print('GLOBAL_MODEL_EPOCHS : ' + str(GLOBAL_MODEL_EPOCHS))
  print('FEDERATED : ' + str(FEDERATED))
  print('GLOBAL_FEDERATED_EPOCHS : ' + str(GLOBAL_FEDERATED_EPOCHS))
  print('CLIENT_EPOCHS : ' + str(CLIENT_EPOCHS))
  print('TOTAL_FEDERATED_CLIENTS : ' + str(TOTAL_FEDERATED_CLIENTS))
  print('ACTIVE_FEDERATED_CLIENTS : ' + str(ACTIVE_FEDERATED_CLIENTS))
  print('BEST_MAPS : ' + str(BEST_MAPS))
  print('GLOBAL_AVG_EPOCHS : ' + str(GLOBAL_AVG_EPOCHS))

  # Initialize the global model YOLOv8 for object detection
  global_model = YOLO('yolov8n.pt')

  # Train the global model
  results = global_model.train(data=f'datasets/{DATASET}/global/data.yaml',
                               epochs=GLOBAL_MODEL_EPOCHS, name='global0',
                               project=f'runs/{PROJECT}')

  # Print the mAP
  print('global0 mAP = ' + str(results.box.map))

  if not FEDERATED:
    return

  # Initialize the random number generator
  rng = random.default_rng()

  #Start the federated learning
  for epoch in range(1, GLOBAL_FEDERATED_EPOCHS + 1):
    print(f'\n | Global Training Round : {epoch} |\n')

    # Array of tuples (client, client mAP)
    maps = []

    # Select the clients
    clients = rng.choice(TOTAL_FEDERATED_CLIENTS, ACTIVE_FEDERATED_CLIENTS,
                         replace=False)
    clients.sort()
    print('Clients : ' + str(clients))

    # Train the clients
    for client in clients:
      # Set up the client model
      client_model = YOLO(f'runs/{PROJECT}/global{epoch - 1}/weights/best.pt')

      # Train the client model
      client_res = client_model.train(data=f'datasets/{DATASET}/client{client}/data.yaml',
                                      epochs=CLIENT_EPOCHS, name=f'client{epoch}{client}',
                                      project=f'runs/{PROJECT}')

      # Print the client mAP
      print(f'client{epoch}{client} mAP = ' + str(client_res.box.map))

      # Save the mAP
      maps.append((client, client_res.box.map))

    clients_weights = []

    # Sort by mAP and take the n best
    maps.sort(key=lambda tup: tup[1], reverse=True)
    for i in range(BEST_MAPS):
      client_model = YOLO(f'runs/{PROJECT}/client{epoch}{maps[i][0]}/weights/best.pt')
      clients_weights.append(client_model.state_dict())
      print(f'chosen client{epoch}{maps[i][0]} mAP = ' + str(maps[i][1]))

    # Average clients best weights
    avg_weights = average_weights(clients_weights)

    #Load the global model and train to test and save the average weights
    global_model = YOLO(f'runs/{PROJECT}/global{epoch - 1}/weights/best.pt')
    global_model.load_state_dict(avg_weights)
    results = global_model.train(data=f'datasets/{DATASET}/global/data.yaml',
                                 epochs=GLOBAL_AVG_EPOCHS, name=f'global{epoch}',
                                 project=f'runs/{PROJECT}')
    print('global' + str(epoch) + ' mAP = ' + str(results.box.map))

try:
  main()
finally:
  # Restart the runtime to clean the memory
  exit()

/content
PROJECT : augmented1500_random
DATASET : augmented1500
GLOBAL_MODEL_EPOCHS : 5
FEDERATED : True
GLOBAL_FEDERATED_EPOCHS : 5
CLIENT_EPOCHS : 20
TOTAL_FEDERATED_CLIENTS : 5
ACTIVE_FEDERATED_CLIENTS : 3
BEST_MAPS : 3
GLOBAL_AVG_EPOCHS : 1
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=datasets/augmented1500/global/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/augmented1500_random, name=global0, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=Fals

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
train: Scanning /content/datasets/augmented1500/global/train/labels.cache... 1200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1200/1200 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/datasets/augmented1500/global/valid/labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]


Plotting labels to runs/augmented1500_random/global0/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/augmented1500_random/global0
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

Plotting labels to runs/augmented1500_random/global5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/augmented1500_random/global5
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      2.34G     0.8777     0.6537      1.071         55        640: 100%|██████████| 75/75 [00:10<00:00,  7.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  2.66it/s]


                   all        300        796      0.856      0.727      0.842      0.587

1 epochs completed in 0.039 hours.
Optimizer stripped from runs/augmented1500_random/global5/weights/last.pt, 6.2MB
Optimizer stripped from runs/augmented1500_random/global5/weights/best.pt, 6.2MB

Validating runs/augmented1500_random/global5/weights/best.pt...
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:03<00:00,  2.80it/s]


                   all        300        796      0.861      0.724      0.842      0.586
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/augmented1500_random/global5
global5 mAP = 0.5862197274370682


## Clean the runtime

Use this to restart the runtime and clean the memory if needed

In [ ]:
exit()

## Zip the runs folder and download it

In [1]:
!zip -r /content/runs.zip /content/runs

from google.colab import files
files.download('/content/runs.zip')

  adding: content/runs/ (stored 0%)
  adding: content/runs/augmented1000_best/ (stored 0%)
  adding: content/runs/augmented1000_best/client22/ (stored 0%)
  adding: content/runs/augmented1000_best/client22/results.csv (deflated 84%)
  adding: content/runs/augmented1000_best/client22/val_batch1_pred.jpg (deflated 5%)
  adding: content/runs/augmented1000_best/client22/labels_correlogram.jpg (deflated 40%)
  adding: content/runs/augmented1000_best/client22/train_batch1.jpg (deflated 2%)
  adding: content/runs/augmented1000_best/client22/F1_curve.png (deflated 16%)
  adding: content/runs/augmented1000_best/client22/train_batch122.jpg (deflated 6%)
  adding: content/runs/augmented1000_best/client22/val_batch1_labels.jpg (deflated 5%)
  adding: content/runs/augmented1000_best/client22/R_curve.png (deflated 15%)
  adding: content/runs/augmented1000_best/client22/val_batch0_labels.jpg (deflated 5%)
  adding: content/runs/augmented1000_best/client22/train_batch121.jpg (deflated 4%)
  adding: co

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>